In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('nghuyong/ernie-1.0')

Some weights of the model checkpoint at /Users/bytedance/.cache/torch/sentence_transformers/nghuyong_ernie-1.0 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Two lists of sentences
sentences1 = ['火把接力传递',
             '火把接力传递',
             '火把接力传递']

sentences2 = ['意思是给它一把火烧光，指用火烧毁',
              '意思是无处安身，四处流浪',
              '意思为焚烧毁坏']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

火把接力传递 		 意思是给它一把火烧光，指用火烧毁 		 Score: 0.7910
火把接力传递 		 意思是无处安身，四处流浪 		 Score: 0.6727
火把接力传递 		 意思为焚烧毁坏 		 Score: 0.7152


In [7]:
import pandas as pd
import json
wiki_info = json.load(open('data/wiki_info_v2.json'))
train_csv = pd.read_csv('data/train.csv')
valid_csv = pd.read_csv('data/val.csv')

In [8]:
def preprocess(data, wiki_info):
    questions = []
    contexts = []
    labels = []
    for idx, row in data.iterrows():
        questions.append(f'{row["riddle"]}')
        labels.append(int(row['label']))
        context = []
        for i in range(5):
            name = f'choice{i}'
            explanation = wiki_info.get(row[name], '')
            context.append(explanation)
        contexts.append(context)
    return questions, contexts, labels

questions, contexts, labels = preprocess(valid_csv, wiki_info)

In [13]:
tp = 0

for context, question, label in zip(contexts, questions, labels):
    pred = 0
    val = 0
    for idx, text in enumerate(context):
        embeddings1 = model.encode(question, convert_to_tensor=True)
        embeddings2 = model.encode(text, convert_to_tensor=True)
        score = util.pytorch_cos_sim(embeddings1, embeddings2)
        if score > val:
            pred = idx
            val = score
    tp += (pred == label)


tensor([[0.7975]])
tensor([[0.7455]])
tensor([[0.7235]])
tensor([[0.6748]])
tensor([[0.7766]])
tensor([[0.7946]])
tensor([[0.7972]])
tensor([[0.6897]])
tensor([[0.7861]])
tensor([[0.7462]])
tensor([[0.7173]])
tensor([[0.7490]])
tensor([[0.6997]])
tensor([[0.7245]])
tensor([[0.7334]])
tensor([[0.7965]])
tensor([[0.7963]])
tensor([[0.8142]])
tensor([[0.8039]])
tensor([[0.7886]])
tensor([[0.8527]])
tensor([[0.8385]])
tensor([[0.8214]])
tensor([[0.7936]])
tensor([[0.8051]])
tensor([[0.8336]])
tensor([[0.8473]])
tensor([[0.8255]])
tensor([[0.8632]])
tensor([[0.8509]])
tensor([[0.8445]])
tensor([[0.8092]])
tensor([[0.8192]])
tensor([[0.8080]])
tensor([[0.8400]])
tensor([[0.8122]])
tensor([[0.7877]])
tensor([[0.8024]])
tensor([[0.7919]])
tensor([[0.8442]])
tensor([[0.7709]])
tensor([[0.7852]])
tensor([[0.7822]])
tensor([[0.7746]])
tensor([[0.7600]])
tensor([[0.6198]])
tensor([[0.5840]])
tensor([[0.7055]])
tensor([[0.7192]])
tensor([[0.7346]])
tensor([[0.7734]])
tensor([[0.7501]])
tensor([[0.7

In [14]:
tp

5